##### Downloading the dataset

In [1]:
import json
import os
import urllib
import torch

In [2]:
%pwd
%cd Instruction_Fine_Tuning

[Errno 2] No such file or directory: 'Instruction_Fine_Tuning'
/Users/mukulagarwal/Desktop/Projects/transformers/Instruction_Fine_Tuning


/opt/miniconda3/envs/torch/lib/python3.12/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [3]:
def download_and_load_file(file_path, url):

    if not os.path.exists(file_path):
        with urllib.request.urlopen(url) as response:
            text_data = response.read().decode("utf-8")
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(text_data)
    else:
        with open(file_path, "r", encoding="utf-8") as file:
            text_data = file.read()

    with open(file_path, "r", encoding="utf-8") as file:
        data = json.load(file)

    return data

In [4]:
file_path = "instruction-data.json"
url = (
    "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch"
    "/main/ch07/01_main-chapter-code/instruction-data.json"
)

data = download_and_load_file(file_path, url)
print("Number of entries:", len(data))

Number of entries: 1100


In [5]:
data[50]

{'instruction': 'Identify the correct spelling of the following word.',
 'input': 'Ocassion',
 'output': "The correct spelling is 'Occasion.'"}

In [6]:
def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )
    
    input_text = f"\n\n### Input:\n{entry['input']}" if entry['input'] else ''
    return instruction_text + input_text

In [7]:
def return_response(entry):
    response = f"\n\n### Response:\n{entry['output']}"
    return response

print(format_input(data[50]) + return_response(data[50]))

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Identify the correct spelling of the following word.

### Input:
Ocassion

### Response:
The correct spelling is 'Occasion.'


Splitting the Dataset

In [8]:
train_portion = int(len(data) * 0.85)
test_portion = int(len(data) * 0.10)

train_data = data[:train_portion]
test_data = data[train_portion:train_portion+test_portion]
validation_data = data[train_portion+test_portion:]

print(len(train_data))
print(len(test_data))
print(len(validation_data))

935
110
55


In [9]:
from torch.utils.data import Dataset,DataLoader

class InstructionDataset(Dataset):
    def __init__(self,data,tokenizer):
        self.data = data
        self.encoded_texts = []
        for entry in self.data:
            instruction_plus_input = format_input(entry)
            response = f"\n\n### Response:\n{entry['output']}"
            full_text = instruction_plus_input + response
            self.encoded_texts.append(tokenizer.encode(full_text))
            
    def __getitem__(self, index):
        return self.encoded_texts[index]
    
    def __len__(self):
        return len(self.data)

In [10]:
def custom_collate_fn(batch,pad_token = 50256,device = "cpu",ignore_index = -100,allowed_max_length = None):
    input_lst, target_lst = [],[]
    batch_max_length = max([len(input_tokens)+1 for input_tokens in batch])
    for item in batch:
        new_item = item.copy()
        new_item += [pad_token]
        
        padded = new_item + [pad_token]*(batch_max_length-len(new_item)) 
        input_padded = torch.tensor(padded[:-1])
        target_padded = torch.tensor(padded[1:])
        
        mask = (target_padded==pad_token)
        indices = torch.nonzero(mask).squeeze()
        if indices.numel() > 1:
            target_padded[indices[1:]] = ignore_index
            
        if allowed_max_length is not None:
            target_padded = target_padded[:allowed_max_length]
            input_padded = input_padded[:allowed_max_length]
        
        input_lst.append(input_padded)
        target_lst.append(target_padded)
        
    input_lst = torch.stack(input_lst).to(device)
    target_lst = torch.stack(target_lst).to(device)
    
    return input_lst,target_lst
    

In [11]:
inputs_1 = [0, 1, 2, 3, 4]
inputs_2 = [5, 6]
inputs_3 = [7, 8, 9]
batch = (
    inputs_1,
    inputs_2,
    inputs_3
)
custom_collate_fn(batch)

(tensor([[    0,     1,     2,     3,     4],
         [    5,     6, 50256, 50256, 50256],
         [    7,     8,     9, 50256, 50256]]),
 tensor([[    1,     2,     3,     4, 50256],
         [    6, 50256,  -100,  -100,  -100],
         [    8,     9, 50256,  -100,  -100]]))

In [12]:
# if torch.backends.mps.is_available():
#     device = torch.device("mps")
    
# print(device)

In [13]:
device = torch.device("cpu")

In [14]:
from functools import partial

customized_collate_func = partial(
    custom_collate_fn,
    device = device,
    allowed_max_length = 1024
)

In [15]:
from torch.utils.data import DataLoader
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

num_workers = 0
batch_size = 8
torch.manual_seed(123)

train_dataset = InstructionDataset(train_data,tokenizer)
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size = batch_size,
    collate_fn=customized_collate_func,
    shuffle=True,
    drop_last=True,
    num_workers=num_workers
)
val_dataset = InstructionDataset(validation_data,tokenizer)
val_loader = DataLoader(
    dataset=val_dataset,
    batch_size = batch_size,
    collate_fn=customized_collate_func,
    shuffle=False,
    drop_last=False,
    num_workers=num_workers
)

test_dataset = InstructionDataset(test_data,tokenizer)
val_loader = DataLoader(
    dataset=test_dataset,
    batch_size = batch_size,
    collate_fn=customized_collate_func,
    shuffle=False,
    drop_last=False,
    num_workers=num_workers
)

In [16]:
print("Validation Loader")
for input_token,target_tokens in val_loader:
    print(input_token.size(),target_tokens.size())

Validation Loader
torch.Size([8, 64]) torch.Size([8, 64])
torch.Size([8, 83]) torch.Size([8, 83])
torch.Size([8, 62]) torch.Size([8, 62])
torch.Size([8, 58]) torch.Size([8, 58])
torch.Size([8, 66]) torch.Size([8, 66])
torch.Size([8, 63]) torch.Size([8, 63])
torch.Size([8, 69]) torch.Size([8, 69])
torch.Size([8, 67]) torch.Size([8, 67])
torch.Size([8, 72]) torch.Size([8, 72])
torch.Size([8, 73]) torch.Size([8, 73])
torch.Size([8, 68]) torch.Size([8, 68])
torch.Size([8, 65]) torch.Size([8, 65])
torch.Size([8, 85]) torch.Size([8, 85])
torch.Size([6, 76]) torch.Size([6, 76])


Loading the Pretrained Model

In [17]:
from transformers import GPT2Model

model_names = {
    "gpt2-small (124M)": "openai-community/gpt2",
    "gpt2-medium (355M)": "openai-community/gpt2-medium",
    "gpt2-large (774M)": "openai-community/gpt2-large",
    "gpt2-xl (1558M)": "openai-community/gpt2-xl"
}

CHOOSE_MODEL = "gpt2-medium (355M)"

gpt_hf = GPT2Model.from_pretrained(model_names[CHOOSE_MODEL],cache_dir='checkpoints')
gpt_hf.eval()

/opt/miniconda3/envs/torch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GPT2Model(
  (wte): Embedding(50257, 1024)
  (wpe): Embedding(1024, 1024)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-23): 24 x GPT2Block(
      (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2SdpaAttention(
        (c_attn): Conv1D(nf=3072, nx=1024)
        (c_proj): Conv1D(nf=1024, nx=1024)
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D(nf=4096, nx=1024)
        (c_proj): Conv1D(nf=1024, nx=4096)
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
)

In [18]:
BASE_CONFIG = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "drop_rate": 0.0,       # Dropout rate
    "qkv_bias": True        # Query-key-value bias
}

model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}


BASE_CONFIG.update(model_configs[CHOOSE_MODEL])

In [19]:
def assign_check(left, right):
    if left.shape != right.shape:
        raise ValueError(f"Shape mismatch. Left: {left.shape}, Right: {right.shape}")
    return torch.nn.Parameter(right.clone().detach())

import numpy as np

def load_weights(gpt, gpt_hf):

    d = gpt_hf.state_dict()

    gpt.pos_emb.weight = assign_check(gpt.pos_emb.weight, d["wpe.weight"])
    gpt.tok_emb.weight = assign_check(gpt.tok_emb.weight, d["wte.weight"])
    
    for b in range(BASE_CONFIG["n_layers"]):
        q_w, k_w, v_w = np.split(d[f"h.{b}.attn.c_attn.weight"], 3, axis=-1)
        gpt.trf_blocks[b].att.W_query.weight = assign_check(gpt.trf_blocks[b].att.W_query.weight, q_w.T)
        gpt.trf_blocks[b].att.W_key.weight = assign_check(gpt.trf_blocks[b].att.W_key.weight, k_w.T)
        gpt.trf_blocks[b].att.W_value.weight = assign_check(gpt.trf_blocks[b].att.W_value.weight, v_w.T)
    
        q_b, k_b, v_b = np.split(d[f"h.{b}.attn.c_attn.bias"], 3, axis=-1)
        gpt.trf_blocks[b].att.W_query.bias = assign_check(gpt.trf_blocks[b].att.W_query.bias, q_b)
        gpt.trf_blocks[b].att.W_key.bias = assign_check(gpt.trf_blocks[b].att.W_key.bias, k_b)
        gpt.trf_blocks[b].att.W_value.bias = assign_check(gpt.trf_blocks[b].att.W_value.bias, v_b)
    
    
        gpt.trf_blocks[b].att.out_proj.weight = assign_check(gpt.trf_blocks[b].att.out_proj.weight, d[f"h.{b}.attn.c_proj.weight"].T)
        gpt.trf_blocks[b].att.out_proj.bias = assign_check(gpt.trf_blocks[b].att.out_proj.bias, d[f"h.{b}.attn.c_proj.bias"])
    
        gpt.trf_blocks[b].ff.layers[0].weight = assign_check(gpt.trf_blocks[b].ff.layers[0].weight, d[f"h.{b}.mlp.c_fc.weight"].T)
        gpt.trf_blocks[b].ff.layers[0].bias = assign_check(gpt.trf_blocks[b].ff.layers[0].bias, d[f"h.{b}.mlp.c_fc.bias"])
        gpt.trf_blocks[b].ff.layers[2].weight = assign_check(gpt.trf_blocks[b].ff.layers[2].weight, d[f"h.{b}.mlp.c_proj.weight"].T)
        gpt.trf_blocks[b].ff.layers[2].bias = assign_check(gpt.trf_blocks[b].ff.layers[2].bias, d[f"h.{b}.mlp.c_proj.bias"])
    
        gpt.trf_blocks[b].norm1.scale = assign_check(gpt.trf_blocks[b].norm1.scale, d[f"h.{b}.ln_1.weight"])
        gpt.trf_blocks[b].norm1.shift = assign_check(gpt.trf_blocks[b].norm1.shift, d[f"h.{b}.ln_1.bias"])
        gpt.trf_blocks[b].norm2.scale = assign_check(gpt.trf_blocks[b].norm2.scale, d[f"h.{b}.ln_2.weight"])
        gpt.trf_blocks[b].norm2.shift = assign_check(gpt.trf_blocks[b].norm2.shift, d[f"h.{b}.ln_2.bias"])
    
        gpt.final_norm.scale = assign_check(gpt.final_norm.scale, d[f"ln_f.weight"])
        gpt.final_norm.shift = assign_check(gpt.final_norm.shift, d[f"ln_f.bias"])
        gpt.out_head.weight = assign_check(gpt.out_head.weight, d["wte.weight"])

In [20]:
%pwd
%cd ..

/Users/mukulagarwal/Desktop/Projects/transformers


/opt/miniconda3/envs/torch/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [21]:
from making_LLM_from_scratch.Scripts.gpt_archietecture import GPTModel
gpt = GPTModel(BASE_CONFIG)

load_weights(gpt, gpt_hf)

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])


In [22]:
from making_LLM_from_scratch.Scripts.training_gpt import training_gpt, calc_loss_loader
gpt.to(device)
torch.manual_seed(123)

with torch.no_grad():
    train_loss = calc_loss_loader(train_loader,gpt,device,num_batches=5)
    val_loss = calc_loss_loader(val_loader,gpt,device,num_batches=5)
    
print(train_loss)
print(val_loss)

3.8258938789367676
3.9149906635284424


In [24]:
PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0
import time

import torch.optim.optimizer
start_time = time.time()
torch.manual_seed(123)

device = torch.device("cpu")

optimizer = torch.optim.AdamW(
    gpt.parameters(),lr = 0.00005, weight_decay=0.1
)
num_epochs = 2
train_losses, val_losses, tokens_seen = training_gpt( gpt, train_loader, val_loader, optimizer, 
                                                     device, num_epochs=num_epochs, eval_freq=5, 
                                                     eval_iter=5, 
                                                     start_context=format_input(validation_data[0]), 
                                                     tokenizer=tokenizer)
end_time = time.time()
execution_time_minutes = (start_time - end_time)/60
print(f"Training completed in {execution_time_minutes:.2f} minutes.")

Ep 1 (Step 000000): Train loss 0.613, Val loss 0.743
Ep 1 (Step 000005): Train loss 0.612, Val loss 0.763
Ep 1 (Step 000010): Train loss 0.610, Val loss 0.773
Ep 1 (Step 000015): Train loss 0.619, Val loss 0.794
Ep 1 (Step 000020): Train loss 0.608, Val loss 0.789
Ep 1 (Step 000025): Train loss 0.592, Val loss 0.780
Ep 1 (Step 000030): Train loss 0.575, Val loss 0.765
Ep 1 (Step 000035): Train loss 0.572, Val loss 0.777
Ep 1 (Step 000040): Train loss 0.563, Val loss 0.790
Ep 1 (Step 000045): Train loss 0.552, Val loss 0.785
Ep 1 (Step 000050): Train loss 0.538, Val loss 0.768
Ep 1 (Step 000055): Train loss 0.528, Val loss 0.752
Ep 1 (Step 000060): Train loss 0.518, Val loss 0.739
Ep 1 (Step 000065): Train loss 0.504, Val loss 0.724
Ep 1 (Step 000070): Train loss 0.498, Val loss 0.713
Ep 1 (Step 000075): Train loss 0.491, Val loss 0.712
Ep 1 (Step 000080): Train loss 0.483, Val loss 0.710
Ep 1 (Step 000085): Train loss 0.471, Val loss 0.702
Ep 1 (Step 000090): Train loss 0.457, Val loss

KeyboardInterrupt: 

In [ ]:
from tqdm import tqdm
from making_LLM_from_scratch.Scripts.training_gpt import text_to_token_ids,token_ids_to_text,generate_simple_text
for i,entry in tqdm(enumerate(test_data),total = len(test_data)):
    instruction = format_input(entry)
    
    response_token_ids = generate_simple_text(gpt,text_to_token_ids(instruction,tokenizer),
                                              max_tokens = 256,
                                              content_size=1024)
    
    response_text = token_ids_to_text(response_token_ids,tokenizer)
    responce_text = responce_text[len(instruction):].replace("### Response","").strip()
    
    test_data[i]['model_response'] = response_text
    
    with open('instruction-data-with-response.json',"w") as file:
        json.dump(test_data, file, indent=4)